In [ ]:
# !pip install git+https://github.com/LIAAD/yake
# !pip install PyMuPDF Pillow

In [1]:
import yake
text=''

In [2]:
import fitz
import io
from PIL import Image
import numpy as np
import pytesseract

In [12]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Tesseract-OCR'

In [3]:
def getKeyWords(text):
    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 20

    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    
    return keywords

In [4]:
def getImgDescp(filename):
    pdf_file = fitz.open(filename)
    pdf_image_list=[]
    imgdescps=[]

    for page_index in range(len(pdf_file)):
    
        page = pdf_file[page_index]
        image_list = pdf_file.get_page_images(page_index)
        if image_list:
            pdf_image_list.append(image_list)
            
        for image_index, img in enumerate(pdf_file.get_page_images(page), start=1):
            xref = img[0]
            pix = fitz.Pixmap(pdf_file,xref)
            piximg = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            # piximgarr = np.array(pix)
            decp=''
            decp=pytesseract.image_to_string(piximg)
            imgdescps.append(decp)
            # base_image = pdf_file.extract_image(xref)
            # image_bytes = base_image['image']
            
            # image_ext = base_image['ext']
            
            
    return imgdescps

In [5]:
import spacy
import pytextrank

d:\PYTHON\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def getsummary(text):
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('textrank')
    doc = nlp(text)
    summ1 = list(doc._.textrank.summary())
    tostring = lambda sent:sent.text
    summary = list(map(tostring,summ1))
    summary = ' '.join(summary)
    return summary

In [7]:
import PyPDF2

In [14]:
def getabstract(filereader):
    fulltext = []
    for i in filereader.pages:
        fulltext+= [i.extract_text()]
        currpage = i.extract_text().lower()
        if 'abstract' in currpage:
            abstract = currpage[currpage.find('abstract'):]
            # print('abstrat',abstract)
            return abstract
        if 'a b s t r a c t' in currpage:
            abstract = currpage[currpage.find('a b s t r a c t'):]
            # print('abstract',abstract)
            return abstract
    
    return ' '.join(fulltext[:4])
    

In [19]:
def main(filename):
    text=''
    # filename=''
    file = open(filename,'rb')
    filereader = PyPDF2.PdfReader(file)
    text = getabstract(filereader)
    resp = {'summary':'','keywords':'',"img_descp":''}
    resp['keywords'] = getKeyWords(text)
    # print(resp['keywords'],'\n\n')
    resp['summary']= getsummary(text)
    print(resp['summary'])
    resp['img_descp'] = getImgDescp(filename)
    return resp

In [20]:
main('./Active-referral-plus-a-small-financial-incentive-upon-c_2021_The-Lancet-Regi.pdf')

{'summary': 'funding:  hong kong council on smoking  and health. \n published  by elsevier  ltd. \nthis is an open access article under the cc by-nc-nd  license \n( http://creativecommons.org/licenses/by-nc-nd/4.0/  ) \n∗corresponding  author : man ping wang, phd, school of nursing, the university  \nof hong kong, 21 sassoon road, pokfulam,  hong kong sar, china \ne-mail address: mpwang@hku.hk  (m.p. wang). \n a b s t r a c t \nbackground:  many smokers  do not use existing  free or low-cost  smoking  cessation  services,  cost-effective  \ninterventions  to increase  use are needed.  \nmethods:  we did a 2-armed  cluster randomised  controlled  trial (crct) in hong kong, china, to evaluate  \nthe effectiveness  of active referral plus a small ﬁnancial  incentive  on abstinence.   the intervention  group received  an offer \nof active referral to cessation  services  at baseline  plus an incentive  (hk$300/us$38)  after using any ces- \nsation services  within 3 months.  ',
 'keywords'